In [1]:
import chromadb

In [25]:
chroma_client = chromadb.PersistentClient(path='./vectordb')

In [26]:
collection = chroma_client.get_or_create_collection(name="documents")

In [27]:
collection.add(
  documents=[
    "this is a document about pineapple",
    "this is a document about oranges"
  ],
  ids=["id1", "id2"]
)

In [5]:
from pprint import pprint
results = collection.query(
  query_texts=["This is a query document about hawaii"], # chroma will emvbed this for me
  n_results =2,  # number of results to return
   where_document={'$contains' : 'pineapple'} 
)
pprint(results)

{'data': None,
 'distances': [[1.0404009819030762]],
 'documents': [['this is a document about pineapple']],
 'embeddings': None,
 'ids': [['id1']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[None]],
 'uris': None}


In [6]:
import polars as pl
articles = pl.read_csv('./archive/Articles.csv', encoding='ISO-8859-1').with_row_index(offset=1)
articles.head()

index,Article,Date,Heading,NewsType
u32,str,str,str,str
1,"""KARACHI: The Sindh government …","""1/1/2015""","""sindh govt decides to cut publ…","""business"""
2,"""HONG KONG: Asian markets start…","""1/2/2015""","""asia stocks up in new year tra…","""business"""
3,"""HONG KONG: Hong Kong shares o…","""1/5/2015""","""hong kong stocks open 0.66 per…","""business"""
4,"""HONG KONG: Asian markets tumbl…","""1/6/2015""","""asian stocks sink euro near ni…","""business"""
5,"""NEW YORK: US oil prices Monday…","""1/6/2015""","""us oil prices slip below 50 a …","""business"""


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

texts = ["Hello world"]
embeddings = model.encode(texts)

print("Embedding shape:", embeddings.shape)  # (1, 384)


/home/mcqueen/Documents/chroma-playground/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding shape: (1, 384)


In [8]:
len(articles)

2692

In [9]:
N = 50
articles_demo = articles[:N]

In [10]:
articles_list = articles_demo['Article'][:].to_list()

In [11]:
vectors = model.encode(articles_list)

In [14]:
ids = [f"id{x}" for x in articles_demo['index'][:].to_list()]

In [15]:
collection = chroma_client.get_or_create_collection(name="articles")

In [17]:
collection.add(
  documents=articles_list,
  ids=ids,
  embeddings = vectors,
)

In [23]:
articles[23]['Article'][0]

'ISLAMABAD: In a move to give relief to consumers, sources in the Finance Ministry said on Tuesday that the price of petrol and petroleum products are expected to decrease further from February 1.According to sources, the price of petrol is expected to be slashed by Rs 10 per litre, High Speed Diesel by Rs 8.50 per litre, Light Diesel by Rs 11 per litre, HOBC by Rs 14 per litre, and Kerosene by Rs 12 per litre.Global crude oil prices have fallen by 50 percent since June 2014, and to provide consistent relief to consumers, the Pakistan government has decreased the price of petrol by Rs 29 since the last four months and brought the price of Diesel down by Rs 23 in the same time frame.Fuel crisis in the country began last week when Pakistan State Oil (PSO) was forced to slash imports because banks refused to extend any more credit to the government-owned company, which supplies 80 percent of the country´s oil.The shortfall led to long queues of angry motorists at petrol stations, though t

In [24]:
query = 'the price of petrol and petroleum products are expected to?'
query_embeddings = model.encode([query])
collection.query(
  query_embeddings=query_embeddings,
  n_results = 2
)

{'ids': [['id24', 'id29']],
 'embeddings': None,
 'documents': [['ISLAMABAD: In a move to give relief to consumers, sources in the Finance Ministry said on Tuesday that the price of petrol and petroleum products are expected to decrease further from February 1.According to sources, the price of petrol is expected to be slashed by Rs 10 per litre, High Speed Diesel by Rs 8.50 per litre, Light Diesel by Rs 11 per litre, HOBC by Rs 14 per litre, and Kerosene by Rs 12 per litre.Global crude oil prices have fallen by 50 percent since June 2014, and to provide consistent relief to consumers, the Pakistan government has decreased the price of petrol by Rs 29 since the last four months and brought the price of Diesel down by Rs 23 in the same time frame.Fuel crisis in the country began last week when Pakistan State Oil (PSO) was forced to slash imports because banks refused to extend any more credit to the government-owned company, which supplies 80 percent of the country´s oil.The shortfall l